In [1]:
import os,sys
from pmtool.ToolBox import ToolBox
import pandas as pd
from pandas import DataFrame
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb

%matplotlib inline

In [15]:
parameters = {'data_path': r'E:\Internship Mass storage\First 3 datasets\Test', # path to your DICOM data
              'data_type': 'dcm', # original data format: DICOM
              'multi_rts_per_pat': False}   # when False, it will look only for 1 rtstruct in the patient folder, 
                                            # this will speed up the process, 
                                            # if you have more then 1 rtstruct per patient, set it to True
data_dcms = ToolBox(**parameters)

100%|██████████| 10/10 [00:10<00:00,  1.03s/it]


In [16]:
dataset_description = data_dcms.get_dataset_description() 
dataset_description.head(10)

Patients processed: 0it [00:00, ?it/s]


,Modality,SliceThickness,PixelSpacing,SeriesDate,Manufacturer


In [10]:
print('Unique modalities found: ', np.unique(dataset_description.Modality.values)[0])

Unique modalities found:  CT


In [17]:
MRI_dcms = ToolBox(**parameters)
dataset_description = MRI_dcms.get_dataset_description('MRI') 
dataset_description.head(10)

100%|██████████| 10/10 [00:10<00:00,  1.01s/it]
Patients processed: 0it [00:00, ?it/s]


,Manufacturer,SliceThickness,PixelSpacing,StudyDate,MagneticFieldStrength,EchoTime


In [12]:
sb.set(context='poster', style='whitegrid')

study_date = sorted([ 'Nan' if x=='' or x=='NaN' else str(x[0:4]) for x in list(dataset_description['SeriesDate'])])[2:]
conv_kernel =['Nan' if x=='' or x=='NaN' else x for x in list(dataset_description['ConvolutionKernel'])]
tube_current =[-1 if x=='' or x=='NaN' else x for x in list(dataset_description['XRayTubeCurrent'])]
exposure =[-1 if x=='' or x=='NaN' else x for x in list(dataset_description['Exposure'])]
ps = sorted([(x[0]) for x in list(filter(lambda x: x != 'NaN', dataset_description['PixelSpacing'].values))])
sl_th = sorted([str(x)[0:3] for x in list(filter(lambda x: x != 'NaN', dataset_description['SliceThickness'].values))])
figures,descriptions = [study_date,conv_kernel,tube_current,exposure,ps,sl_th],['Study Date','Conv Kernel','Tube Current','Exposure','Pixel spacing','Slice Thickness']

fig,ax = plt.subplots(2,3,figsize=(25,15))
for i in range(2):
    for j in range(3):
        ax[i,j].hist(figures.pop(0),alpha=0.7)
        ax[i,j].set_title(descriptions.pop(0),fontsize=20)

KeyError: 'SeriesDate'

In [ ]:
qc_params = {'specific_modality': 'MRI', # target modality: CT
            'thickness_range': [1,2], # slice thickness should be in range of 2..5 mm (changed to 1-2 for this set)
            'spacing_range': [0.5,1.25], # pixel spacing should be in range of 0.5..1.25 mm
            'scan_length_range': [5,170], # scan should contain from 5 to 170 slices
            'axial_res': [512,512], # the axial resolution should be 512x512
            'kernels_list': ['standard','lung','b19f']} # the following kernels are acceptable

In [ ]:
qc_dataframe = MRI_dcms.get_quality_checks(qc_params)

In [ ]:
qc_dataframe

In [ ]:
export_path = r'C:\Users\MegaMert\Desktop\Internship files\Tutorial data\Converted'
region_of_interest = 'all'
data_ct = ToolBox(**parameters) 


In [ ]:
data_ct.convert_to_nrrd(export_path, 'gtv')

In [ ]:
data_ct_nrrd = ToolBox(data_path = r'C:\Users\MegaMert\Desktop\Internship files\Tutorial data\Converted\Test', data_type='nrrd')


In [ ]:
data_ct_nrrd.get_jpegs(r'C:\Users\MegaMert\Desktop\Internship files\Tutorial data\Converted') # the function will create 'images_quick_check' folder in the specified directory 

In [ ]:
from ipywidgets import interact
import numpy as np
from PIL import Image

def browse_images(images,names):
    n = len(images)
    def view_image(i):
        plt.figure(figsize=(20,10))
        plt.imshow(images[i])#, cmap=plt.cm.gray_r, interpolation='nearest')
        plt.title('Slice: %s' % names[i])
        plt.axis('off')
        plt.show()
    interact(view_image, i=(0,n-1))

for pat,_ in data_ct_nrrd:
    _,file_struct = [*os.walk(os.path.join(r'C:\Users\MegaMert\Desktop\Internship files\Tutorial data\Converted\images_quick_check',pat))]
    root,images = file_struct[0],file_struct[2]
    imgs =[np.array(Image.open(os.path.join(root,img))) for img in images]
    print(pat)
    browse_images(imgs,images)
    break

In [ ]:
parameters = r"C:\Users\MegaMert\Documents\GitHub\precision-medicine-toolbox\examples\example_ct_parameters.yaml"
features = data_ct_nrrd.extract_features(parameters, loggenabled=True)

In [ ]:
sb.set(context='poster', style='white')
data_ct_nrrd.pre_process(ref_img_path = r'C:\Users\MegaMert\Desktop\Internship files\Tutorial data\Converted\converted_nrrds\LUNG1-001_1-1\image.nrrd',
                         save_path = r'C:\Users\MegaMert\Desktop\Internship files\Tutorial data\Converted\Preprocessed',
                         hist_match = False,        # boolean
                         subcateneus_fat = False,   # boolean
                         fat_value = 774,           # this is a dummy value, 
                                                    # you would need to find that value for each image
                         percentile_scaling = False, # boolean
                         window_filtering_params = (1500,-600), #Window Filtering [WW,WL]
                         binning = 255,             # this is a dummy value, it takes False or int 
                                                    # (# of bins for intensity resampling)
                         verbosity = True,          # boolean
                         z_score = True,           # boolean 
                         hist_equalize = True,     # boolean
                         norm_coeff = (1000.,500.), # these are dummy values, you would need to estimate real mu and sigma
                                                    # it takes None or tuple: (mu,sigma)
                         visualize = True)

In [ ]:
features.head(3)

In [ ]:
writer = pd.ExcelWriter(r'C:\Users\MegaMert\Documents\GitHub\precision-medicine-toolbox\data/features/extracted_features.xlsx') 
features.to_excel(writer, 'Sheet1')
writer.save()